In [1]:
from model import Corpus, Utterance, User


In [2]:
def stat_summary(corpus: Corpus) -> None:
    print("Number of conversations:", len(list(corpus.iter_conversations())))
    print("Number of utterances:", len(list(corpus.iter_utterances())))
    print("Number of users:", len(list(corpus.iter_users())))

## Merging two corpora

Let's take a look at the characteristics of our input corpora before the merge. Apart from the summary statistics, notice that User 'foxtrot' appears in both corpora. Moreover, it has User metadata that is inconsistent.

The root field in each Utterance indicates where a new Conversation begins. In this case, while there are 2 conversations in each corpus, 1 conversation (with root 2) is featured in both corpuses, so there are only 3 conversations in total.

### Corpus 1

In [3]:
corpus1 = Corpus(utterances = [
            Utterance(id=0, root=0, text="hello world", user=User(name="alice")),
            Utterance(id=1, root=0, reply_to=0, text="my name is bob", user=User(name="bob")),
            Utterance(id=2, root=2, text="this is a sentence", user=User(name="foxtrot", meta={"yellow": "food"})),
        ])

In [4]:
stat_summary(corpus1)

Number of conversations: 2
Number of utterances: 3
Number of users: 3


### Corpus 2

In [5]:
corpus2 = Corpus(utterances = [
            Utterance(id=3, root=3, text="i like pie", user=User(name="charlie", meta={"what": "a mood", "hey": "food"})),
            Utterance(id=4, root=3, reply_to=3, text="sentence galore", user=User(name="echo")),
            Utterance(id=2, root=2, text="this is a sentence", user=User(name="foxtrot", meta={"yellow": "mood", "hello": "world"})),
        ])

In [6]:
stat_summary(corpus2)

Number of conversations: 2
Number of utterances: 3
Number of users: 3


Let's attempt a merge:

In [7]:
corpus3 = corpus1.merge(corpus2)

In [8]:
stat_summary(corpus3)

Number of conversations: 3
Number of utterances: 5
Number of users: 5


### Merging user metadata

Notice that because User 'foxtrot' had conflicting metadata, the latest utterance (i.e. the utterance in corpus2) had its User metadata for 'foxtrot' take precedence. We verify this below. Note too that the other metadata key-value pair ('hello': 'world') has been added to the metadata as well.

In [9]:
corpus3.get_user('foxtrot').meta

{'yellow': 'mood', 'hello': 'world'}

Users were not initialized with their list of corresponding utterances / conversations. Corpus has a method for updating these User lists.

In [10]:
print(list(corpus3.iter_users()))
user_echo = corpus3.get_user('echo')
print("Number of utterances for User echo:", len(list(user_echo.iter_utterances())))
print("Number of conversations for User echo:", len(list(user_echo.iter_conversations())))

[User([('name', 'bob')]), User([('name', 'echo')]), User([('name', 'foxtrot')]), User([('name', 'charlie')]), User([('name', 'alice')])]
Number of utterances for User echo: 0
Number of conversations for User echo: 0


In [11]:
corpus3.update_users_data()

In [12]:
print("Number of utterances for User echo:", len(list(user_echo.iter_utterances())))
print("Number of conversations for User echo:", len(list(user_echo.iter_conversations())))

Number of utterances for User echo: 1
Number of conversations for User echo: 1


### Merging Utterance and Corpus metadata 

We quickly demonstrate the Utterance and Corpus metadata merging functionality. This is all handled in the merge() function as well, we just make its effects explicit here. In addition, we encode the corpora with problematic data/metadata so that the warning functionality is explicit.

(Note that if Utterances have the same id but different data, the Utterance from the other Corpus is ignored and a warning is printed, though the User metadata is still kept.)

### Corpus 4

In [13]:
corpus4 = Corpus(utterances = [
            Utterance(id=0, root=0, text="hello world", user=User(name="alice"), meta={'in': 'wonderland'}),
            Utterance(id=1, root=0, reply_to=0, text="my name is bob", user=User(name="bob"), meta={'fu': 'bu'})
        ])
corpus4.add_meta('AB', 1)
corpus4.add_meta('CD', 2)


In [14]:
stat_summary(corpus4)

Number of conversations: 1
Number of utterances: 2
Number of users: 2


### Corpus 5

In [15]:
corpus5 = Corpus(utterances = [
            Utterance(id=0, root=0, text="hello world", user=User(name="alice"), meta={'in': 'the hat'}),
            Utterance(id=1, root=0, reply_to=0, text="my name is bobbb", user=User(name="bob"), meta={'barrel': 'roll'})
        ])
corpus5.add_meta('AB', 3)
corpus5.add_meta('EF', 3)

In [16]:
stat_summary(corpus5)

Number of conversations: 1
Number of utterances: 2
Number of users: 2


In [17]:
corpus6 = corpus4.merge(corpus5)

Utterance({'id': 1, 'user': User([('name', 'bob')]), 'root': 0, 'reply_to': 0, 'timestamp': None, 'text': 'my name is bob', 'meta': {'fu': 'bu'}})
Utterance({'id': 1, 'user': User([('name', 'bob')]), 'root': 0, 'reply_to': 0, 'timestamp': None, 'text': 'my name is bobbb', 'meta': {'barrel': 'roll'}})
Ignoring second corpus's utterance.


In [18]:
stat_summary(corpus6)

Number of conversations: 1
Number of utterances: 2
Number of users: 2


In [19]:
corpus6.meta

{'AB': 3, 'CD': 2, 'EF': 3}

In [20]:
corpus6.get_utterance(1)

Utterance({'id': 1, 'user': User([('name', 'bob')]), 'root': 0, 'reply_to': 0, 'timestamp': None, 'text': 'my name is bob', 'meta': {'fu': 'bu'}})

In [21]:
corpus6.get_utterance(0)

Utterance({'id': 0, 'user': User([('name', 'alice')]), 'root': 0, 'reply_to': None, 'timestamp': None, 'text': 'hello world', 'meta': {'in': 'the hat'}})

For the most part however, as long as the data is well behaved (e.g. User/Utterance/Conversation/Corpus do not have different values for the same key in the metadata, Utterances with the same id have the same data) one should expect to see no warnings when using merge().